# Education Data Analysis Project

## Overview
This project presents a comprehensive analysis of education data across different countries. The analysis combines data from multiple sources to examine education investment, quality, outcomes, and resource allocation patterns.

### Project Objectives
1. Analyze education investment trends
2. Evaluate education quality indicators
3. Assess resource allocation efficiency
4. Predict future education metrics

### Analysis Components
- Data collection from Eurostat
- Statistical analysis
- Time series forecasting
- Interactive visualizations

## 1. Environment Setup

First, let's import all necessary packages and set up configurations.

##  Environment Installation

Before starting the analysis, we need to install the required packages. Run the following commands in your terminal or in a notebook cell:

In [3]:
# Install required packages
!pip install pandas numpy plotly scikit-learn statsmodels pymongo psycopg2-binary python-dotenv eurostat tqdm

# Verify installations
import pkg_resources
required_packages = [
    'pandas',
    'numpy',
    'plotly',
    'scikit-learn',
    'statsmodels',
    'pymongo',
    'psycopg2-binary',
    'python-dotenv',
    'eurostat',
    'tqdm'
]

# Check installed versions
for package in required_packages:
    version = pkg_resources.get_distribution(package).version
    print(f"{package}: v{version}")


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
pandas: v2.1.3
numpy: v1.24.3
plotly: v5.24.1
scikit-learn: v1.3.2
statsmodels: v0.14.4
pymongo: v4.6.0
psycopg2-binary: v2.9.10
python-dotenv: v1.0.0
eurostat: v1.0.4
tqdm: v4.66.1


/var/folders/3n/6qgmz73j6lx3gzkfjw5151nh0000gn/T/ipykernel_63881/958489629.py:5: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [1]:
# Import required packages
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
import eurostat
import logging
from datetime import datetime
import os
from dotenv import load_dotenv

# Set up logging
logging.basicConfig(
    filename='education_data_collection.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [4]:
logging.basicConfig(
    filename='education_data_collection.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)


# 3. Database Configuration
load_dotenv()

# PostgreSQL configuration
POSTGRES_CONFIG = {
    'dbname': os.getenv('POSTGRES_DB', 'education_db'),
    'user': os.getenv('POSTGRES_USER'),
    'password': os.getenv('POSTGRES_PASSWORD'),
    'host': os.getenv('POSTGRES_HOST'),
    'port': os.getenv('POSTGRES_PORT')
}

# MongoDB configuration
MONGODB_HOST = os.getenv('MONGODB_HOST')
MONGODB_PORT = int(os.getenv('MONGODB_PORT'))
MONGODB_USER = os.getenv('MONGODB_USER')
MONGODB_PASSWORD = os.getenv('MONGODB_PASSWORD')
MONGODB_DB = os.getenv('MONGODB_DB')


In [5]:
# 4. Database Connection Functions
def get_postgres_connection():
    """Get PostgreSQL connection with retry mechanism"""
    max_retries = 3
    retry_count = 0
    
    while retry_count < max_retries:
        try:
            conn = psycopg2.connect(
                dbname=os.getenv('POSTGRES_DB'),
                user=os.getenv('POSTGRES_USER'),
                password=os.getenv('POSTGRES_PASSWORD'),
                host=os.getenv('POSTGRES_HOST'),
                port=os.getenv('POSTGRES_PORT'),
                connect_timeout=30  # Increase timeout to 30 seconds
            )
            print("Successfully connected to PostgreSQL")
            return conn
        except Exception as e:
            print(f"Attempt {retry_count + 1} failed: {str(e)}")
            retry_count += 1
            if retry_count < max_retries:
                time.sleep(2)  # Wait 2 seconds before retrying
    
    print("Failed to connect to PostgreSQL after all retries")
    return None

def get_mongodb_connection():
    """Get MongoDB connection with retry mechanism"""
    try:
        client = MongoClient(
            host=os.getenv('MONGODB_HOST'),
            port=int(os.getenv('MONGODB_PORT')),
            username=os.getenv('MONGODB_USER'),
            password=os.getenv('MONGODB_PASSWORD'),
            serverSelectionTimeoutMS=30000,  # Increase timeout to 30 seconds
            connectTimeoutMS=30000,
            retryWrites=True,
            w='majority'
        )
        db = client[os.getenv('MONGODB_DB')]
        # Test connection
        client.server_info()
        print("Successfully connected to MongoDB")
        return db
    except Exception as e:
        print(f"Error connecting to MongoDB: {str(e)}")
        return None

In [6]:

# 6. Data Processing and Storage Functions
def process_education_data(df, metric_type):
    """Process education data with error handling and data validation"""
    try:
        # Remove any duplicate columns
        df = df.loc[:, ~df.columns.duplicated()]
        
        # Basic data cleaning
        df = df.dropna(how='all')  # Drop rows where all values are NaN
        
        # Convert numeric columns to float
        numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
        for col in numeric_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # Add metadata
        df['metric_type'] = metric_type
        df['processed_date'] = pd.Timestamp.now()
        
        # Validate data
        if df.empty:
            raise ValueError(f"No valid data found for {metric_type}")
            
        return df
        
    except Exception as e:
        logging.error(f"Error processing {metric_type} data: {str(e)}")
        return None

def store_in_mongodb(db, collection_name, records):
    """Store data in MongoDB with error handling and batch processing"""
    try:
        collection = db[collection_name]
        
        # Use bulk write operations for better performance
        operations = []
        for record in records:
            # Create unique identifier
            filter_dict = {
                'country': record.get('country'),
                'year': record.get('year'),
                'metric_type': record.get('metric_type')
            }
            
            operations.append(
                UpdateOne(
                    filter_dict,
                    {'$set': record},
                    upsert=True
                )
            )
            
        if operations:
            # Process in batches of 1000
            batch_size = 1000
            for i in range(0, len(operations), batch_size):
                batch = operations[i:i + batch_size]
                collection.bulk_write(batch, ordered=False)
                
        return True
        
    except Exception as e:
        logging.error(f"Error storing data in MongoDB: {str(e)}")
        return False

def store_in_postgres(conn, table_name, records):
    """Store data in PostgreSQL with error handling"""
    try:
        with conn.cursor() as cur:
            # Convert records to list of tuples for PostgreSQL
            values = []
            for record in records:
                try:
                    values.append((
                        str(record['country']),
                        int(record['year']),
                        str(record['metric_type']),
                        float(record['value'])
                    ))
                except (ValueError, TypeError) as e:
                    print(f"Skipping record due to conversion error: {e}")
                    continue
            
            if values:
                execute_values(cur, f"""
                    INSERT INTO {table_name} (country, year, metric_name, metric_value)
                    VALUES %s
                    ON CONFLICT (country, year, metric_name) 
                    DO UPDATE SET metric_value = EXCLUDED.metric_value;
                """, values)
                conn.commit()
                print(f"Stored {len(values)} records in PostgreSQL for {table_name}")
                
    except Exception as e:
        print(f"Error storing data in PostgreSQL: {str(e)}")
        conn.rollback()

def collect_eurostat_data(metric_name):
    """Collect education data from Eurostat with error handling"""
    try:
        # Define dataset codes
        dataset_codes = {
            'education_investment': 'educ_uoe_fine09',
            'student_teacher_ratio': 'educ_uoe_perp04',
            'completion_rate': 'edat_lfse_03'
        }
        
        if metric_name not in dataset_codes:
            raise ValueError(f"Unknown metric: {metric_name}")
            
        # Get dataset code
        dataset_code = dataset_codes[metric_name]
        
        # Collect data from Eurostat
        df = eurostat.get_data_df(dataset_code)
        
        if df is None or df.empty:
            print(f"No data found for {metric_name}")
            return None
            
        # Process the DataFrame
        df = df.reset_index()
        
        # Get time column (should contain country codes)
        time_cols = [col for col in df.columns if 'TIME' in col]
        if not time_cols:
            print(f"No time column found for {metric_name}")
            return None
            
        time_col = time_cols[0]
        
        # Extract country from the time column
        df['country'] = df[time_col].str.split('\\').str[0]
        
        # Get year columns (numeric columns)
        year_cols = [col for col in df.columns if str(col).isdigit()]
        if not year_cols:
            print(f"No year columns found for {metric_name}")
            return None
            
        # Create processed dataframe
        processed_data = []
        
        # Process each year
        for year in year_cols:
            year_data = df[['country', year]].copy()
            year_data = year_data.rename(columns={year: 'value'})
            year_data['year'] = int(year)
            processed_data.append(year_data)
            
        # Combine all years
        result_df = pd.concat(processed_data, ignore_index=True)
        
        # Clean up the data
        result_df['value'] = pd.to_numeric(result_df['value'], errors='coerce')
        result_df = result_df.dropna(subset=['value'])
        
        return result_df
        
    except Exception as e:
        print(f"Error collecting {metric_name} data: {str(e)}")
        return None

def collect_and_store_education_data(pg_conn, mongo_db):
    """Collect and store education data with improved error handling"""
    metrics = {
        'education_investment': 'educ_investment',
        'student_teacher_ratio': 'student_teacher',
        'completion_rate': 'completion_rate'
    }
    
    for metric_name, table_name in metrics.items():
        try:
            print(f"Collecting {metric_name} data...")
            
            # Collect data
            df = collect_eurostat_data(metric_name)
            if df is None:
                print(f"Skipping {metric_name} due to data collection error")
                continue
            
            # Add metadata
            df['metric_type'] = metric_name
            df['processed_date'] = pd.Timestamp.now()
            
            # Convert to records
            records = df.to_dict('records')
            
            if not records:
                print(f"No records to store for {metric_name}")
                continue
            
            # Store in PostgreSQL
            try:
                with pg_conn.cursor() as cur:
                    values = [(
                        str(record['country']),
                        int(record['year']),
                        str(metric_name),
                        float(record['value'])
                    ) for record in records if all(k in record for k in ['country', 'year', 'value'])]
                    
                    if values:
                        execute_values(cur, f"""
                            INSERT INTO raw_education_data (country, year, metric_name, metric_value)
                            VALUES %s
                            ON CONFLICT (country, year, metric_name) 
                            DO UPDATE SET metric_value = EXCLUDED.metric_value;
                        """, values)
                        pg_conn.commit()
                        print(f"Stored {len(values)} records in PostgreSQL for {metric_name}")
            except Exception as e:
                print(f"Error storing data in PostgreSQL: {str(e)}")
                pg_conn.rollback()
            
            # Store in MongoDB
            try:
                collection = mongo_db[metric_name]
                for record in records:
                    if all(k in record for k in ['country', 'year', 'value']):
                        collection.update_one(
                            {
                                'country': str(record['country']),
                                'year': int(record['year'])
                            },
                            {
                                '$set': {
                                    'value': float(record['value']),
                                    'metric_type': metric_name,
                                    'updated_at': datetime.now()
                                }
                            },
                            upsert=True
                        )
                print(f"Successfully stored records in MongoDB for {metric_name}")
            except Exception as e:
                print(f"Error storing data in MongoDB: {str(e)}")
            
        except Exception as e:
            print(f"Error processing {metric_name}: {str(e)}")
            continue

## 2. Data Collection

This section collects education-related data from Eurostat, including:
- Education investment
- Student-teacher ratio
- Completion rates
- Literacy rates

In [2]:
def collect_education_data():
    """
    Collect education data from Eurostat
    Returns a dictionary containing different education metrics
    """
    datasets = {
        'education_investment': 'educ_uoe_fine09',  # Education investment
        'student_teacher_ratio': 'educ_uoe_perp04', # Student-teacher ratio
        'completion_rate': 'edat_lfse_03',          # Completion rate
        'literacy_rate': 'edat_lfse_01'            # Literacy rate
    }
    
    data = {}
    for metric, code in datasets.items():
        try:
            logging.info(f"Starting collection of {metric} data...")
            print(f"Collecting {metric} data...")
            
            # Get raw data from Eurostat
            df = eurostat.get_data_df(code)
            print(f"\nColumns in {metric} dataset:")
            print(df.columns.tolist())
            
            # First, handle the time columns
            time_cols = [col for col in df.columns if col.isdigit()]
            
            # Identify non-time columns
            non_time_cols = [col for col in df.columns if not col.isdigit()]
            
            # Create a base DataFrame with non-time columns
            base_df = df[non_time_cols].copy()
            
            # Initialize list to store transformed data
            transformed_data = []
            
            # Process each row
            for idx, row in base_df.iterrows():
                for year in time_cols:
                    new_row = row.copy()
                    new_row['year'] = year
                    new_row[metric] = df.loc[idx, year]
                    transformed_data.append(new_row)
            
            # Create new DataFrame from transformed data
            transformed_df = pd.DataFrame(transformed_data)
            
            # Clean the data
            if 'geo\\TIME_PERIOD' in transformed_df.columns:
                transformed_df['geo'] = transformed_df['geo\\TIME_PERIOD'].str.split('\\').str[0]
                transformed_df = transformed_df.drop('geo\\TIME_PERIOD', axis=1)
            
            # Convert metric values to numeric
            transformed_df[metric] = pd.to_numeric(transformed_df[metric], errors='coerce')
            
            # Store the transformed DataFrame
            data[metric] = transformed_df
            
            print(f"Successfully collected and transformed {metric} data")
            print(f"Shape of {metric} dataset: {transformed_df.shape}")
            print(f"Columns in transformed dataset: {transformed_df.columns.tolist()}\n")
            
        except Exception as e:
            print(f"Error collecting {metric} data: {str(e)}")
            print(f"Full error details: ", e)
            import traceback
            print(traceback.format_exc())
    
    return data

# Execute data collection with more detailed output
print("Starting education data collection...")
education_data = collect_education_data()

# Display the structure of collected data
for metric, df in education_data.items():
    print(f"\nStructure of {metric} dataset:")
    print(df.info())
    print("\nFirst few rows:")
    print(df.head())
    print("\n" + "="*50)

Starting education data collection...

Columns in education_investment dataset:
['freq', 'unit', 'isced11', 'geo\\TIME_PERIOD', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
Successfully collected and transformed education_investment data
Shape of education_investment dataset: (25850, 6)
Columns in transformed dataset: ['freq', 'unit', 'isced11', 'year', 'education_investment', 'geo']


Columns in student_teacher_ratio dataset:
['freq', 'unit', 'isced11', 'geo\\TIME_PERIOD', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']
Successfully collected and transformed student_teacher_ratio data
Shape of student_teacher_ratio dataset: (5960, 6)
Columns in transformed dataset: ['freq', 'unit', 'isced11', 'year', 'student_teacher_ratio', 'geo']


Columns in completion_rate dataset:
['freq', 'sex', 'age', 'unit', 'isced11', 'geo\\TIME_PERIOD', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2

## 3. Data Preprocessing

Clean and standardize the collected data, including:
- Merging different datasets
- Handling missing values
- Standardizing column names
- Converting data types

In [ ]:
def preprocess_data(data):
    """
    Preprocess and merge all education data
    Returns a cleaned and standardized DataFrame
    """
    processed_data = None
    
    for metric, df in data.items():
        print(f"Processing {metric} data...")
        
        # Determine merge columns
        common_cols = ['geo', 'time', 'year']
        merge_cols = [col for col in common_cols if col in df.columns]
        
        if not merge_cols:
            print(f"Warning: No merge columns found in {metric} dataset")
            continue
        
        # Prepare current dataset
        cols_to_keep = merge_cols + [metric]
        current_df = df[cols_to_keep].copy()
        
        # Merge data
        if processed_data is None:
            processed_data = current_df
        else:
            processed_data = processed_data.merge(
                current_df,
                on=merge_cols,
                how='outer'
            )
    
    if processed_data is not None:
        # Clean and standardize data
        processed_data = processed_data.dropna()
        processed_data['year'] = pd.to_numeric(processed_data['year'])
        processed_data = processed_data.rename(columns={'geo': 'country'})
        processed_data = processed_data.sort_values('year')
        
        print("Data preprocessing completed")
    else:
        print("Warning: No valid data after preprocessing")
        processed_data = pd.DataFrame()
    
    return processed_data

# Execute data preprocessing
print("\nStarting data preprocessing...")
processed_education_data = preprocess_data(education_data)

# Display processed data structure
print("\nProcessed data structure:")
print(processed_education_data.info())
print("\nData preview:")
print(processed_education_data.head())


Starting data preprocessing...
Processing education_investment data...
Processing student_teacher_ratio data...
Processing completion_rate data...
Processing literacy_rate data...


## 4. Data Analysis

Perform multi-dimensional analysis on preprocessed data:
- Basic statistical analysis
- Trend analysis
- Country comparisons

In [ ]:
def analyze_education_metrics(df):
    """
    Analyze education indicators
    Returns a dictionary containing analysis results and visualizations
    """
    results = {}
    
    # Basic statistical analysis
    print("Performing basic statistical analysis...")
    for column in df.select_dtypes(include=[np.number]).columns:
        if column != 'year':
            stats = df.groupby('year')[column].agg(['mean', 'std', 'min', 'max'])
            results[f'{column}_stats'] = stats
            print(f"\nBasic statistics for {column}:")
            print(stats)
    
    # Trend analysis
    print("\nPerforming trend analysis...")
    for column in df.select_dtypes(include=[np.number]).columns:
        if column != 'year':
            fig = px.line(df.groupby('year')[column].mean().reset_index(),
                         x='year', y=column,
                         title=f'Trend of {column} Over Time')
            results[f'{column}_trend'] = fig
            fig.show()
    
    # Country comparison
    print("\nPerforming country comparison...")
    latest_year = df['year'].max()
    for column in df.select_dtypes(include=[np.number]).columns:
        if column != 'year':
            latest_data = df[df['year'] == latest_year].sort_values(column, ascending=False)
            fig = px.bar(latest_data.head(10),
                        x='country', y=column,
                        title=f'Top 10 Countries - {column} ({latest_year})')
            results[f'{column}_comparison'] = fig
            fig.show()
    
    return results

# Execute data analysis
print("Starting data analysis...")
analysis_results = analyze_education_metrics(processed_education_data)

## 5. Forecasting Analysis

Perform time series forecasting on education metrics:
- Using SARIMA models
- Generate 5-year predictions
- Visualize forecast results